Access GridMET data from Microsoft Planetary computer using their Stac API. File is Zarr. Access only the data needed, and the variables needed (precipitation only for now). 

Just realized Microsoft Planetary computer only has data through 2020 for this....

In [ ]:
import pystac_client
import planetary_computer
import xarray as xr

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
time_range = "2025-06-24"
bbox = [-126, 24, -65, 50]

asset = catalog.get_collection("gridmet").assets["zarr-abfs"]
asset


In [ ]:
ds = xr.open_zarr(
    asset.href,
    storage_options=asset.extra_fields["xarray:storage_options"],
    **asset.extra_fields["xarray:open_kwargs"]
)
ds

Pulled the 06/24 file manually from the GridMET site (on 06/25). 

Notes:
* Metadata says that the day runs from midnight to midnight Mountain Standard time. This leads to the differences between this and GFS/PRISM. 
* Could use Pygridmet package to access the data, but it seems like it might be easier to just use the netcdf file. 


In [ ]:
import xarray as xr

ds = xr.open_dataset("data/gridmet/pr_2025_gridmet.nc", engine="netcdf4")
ds

In [ ]:
ds_pr_0623 = ds.sel(day='2025-06-23T00:00:00.000000000')
ds_pr_0623['precipitation_amount'].attrs

In [ ]:
ds_pr_0623['precipitation_amount'].plot(robust=True)

Attempt to download file froom gridmet site using requests. 

In [ ]:
import requests

In [ ]:
url = "http://www.northwestknowledge.net/metdata/data/pr_2025.nc"

In [ ]:
response = requests.get(url, stream=True)

In [ ]:
local_filename = "data/gridmet/pr_2025_gridmet.nc"

response.raise_for_status()  # Raise an error for bad status codes
with open(local_filename, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:  # Skip keep-alive chunks
            f.write(chunk)

In [ ]:
import xarray as xr

ds_gridmet = xr.open_dataset("data/gridmet/pr_2025_gridmet.nc", engine="netcdf4")
ds_gridmet

In [ ]:
ds_gridmet_0623 = ds_gridmet.sel(day='2025-06-23T00:00:00.000000000', method="nearest")
ds_gridmet_0623

In production data pull, use method="nearest" to get most recent day (with today - 1 as starting point)

In [ ]:
ds_gridmet_0623['precipitation_amount'].plot(robust=True)

In [ ]:
import rioxarray as rxr

ds_gridmet_0623.crs

In [ ]:
# Extract GeoTransform values and convert to Affine
from rasterio.transform import Affine

gt_vals = [
    -124.7666666333333,  # origin x
    0.041666666666666,   # pixel width
    0.0,                 # x rotation
    49.400000000000000,  # origin y
    0.0,                 # y rotation
    -0.041666666666666   # pixel height (negative because top-to-bottom)
]

transform = Affine.from_gdal(*gt_vals)

# Assign transform and CRS manually
ds_gridmet_0623 = ds_gridmet_0623.rio.write_transform(transform)
ds_gridmet_0623 = ds_gridmet_0623.rio.write_crs("EPSG:4326")  # GridMET is in lat/lon WGS84

# Now reproject to EPSG:5070 (Albers Equal Area for CONUS)
ds_gridmet_0623_5070 = ds_gridmet_0623.rio.reproject("EPSG:5070")

In [ ]:
ds_gridmet_0623_5070 = ds_gridmet_0623_5070['precipitation_amount']

In [ ]:
ds_gridmet_0623["precipitation_amount"].plot()

In [ ]:
ds_gridmet_0623_5070.plot(robust=True)

In [ ]:
ds_gridmet_0623_5070.to_netcdf("data/gridmet/pr_gridmet_5070_06232025.nc")

Gridmet precip data is in mm